In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

#Prueba usando LightGBM con algunas variables categoricas

In [30]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

train_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/train_set_Features.csv')
test_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/test_set_Features.csv')

In [31]:
train_set_features = train_set_features.drop_duplicates(subset="Opportunity_ID")
test_set_features = test_set_features.drop_duplicates(subset="Opportunity_ID")

In [32]:
train_set_features['Account_Owner'] = pd.Series(train_set_features['Account_Owner'], dtype="category")
train_set_features['Opportunity_Owner'] = pd.Series(train_set_features['Opportunity_Owner'], dtype="category")
train_set_features['Stage'] = pd.Series(train_set_features['Stage'], dtype="category")

In [33]:
test_set_features['Account_Owner'] = pd.Series(test_set_features['Account_Owner'], dtype="category")
test_set_features['Opportunity_Owner'] = pd.Series(test_set_features['Opportunity_Owner'], dtype="category")

In [34]:
#X = DATA , Y = TARGET 
X1, y1 = train_set_features[['Account_Owner',
                            'Opportunity_Owner',
                            'Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']], train_set_features['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

In [36]:
import lightgbm as lgb

In [37]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = [0,1],)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

Usamos como técnica de boosting: gbdt (gradient boosted decision trees).

In [38]:
lgb_clas = lgb.LGBMClassifier(boosting_type = 'gbdt', 
                                   eval_metric = 'logloss',
                                   verbose_eval = 20,
                              early_stopping_rounds = 3,
                              n_jobs=4)

###Probamos usando Random Search

In [28]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

posibles_hiperparametros = {'learning_rate':np.arange(0.001,0.3,0.001),
                              'n_estimators':np.arange(16,116,5),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(2,7,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.4,0.8,0.1),
                            'colsample_bytree':np.arange(0.3,0.8,0.05),
                              'colsample_bylevel':np.arange(0.3,0.8,0.05)}

random_hiperparametros = RandomizedSearchCV(lgb_clas, 
                                            posibles_hiperparametros, 
                                            n_iter = 100)

mejores_hp = random_hiperparametros.fit(X_train, y_train, eval_set=(X_train, y_train))

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Se truncaron las últimas líneas 5000 del resultado de transmisión.
[44]	valid_0's multi_logloss: 0.401219
[45]	valid_0's multi_logloss: 0.399458
[46]	valid_0's multi_logloss: 0.39739
[47]	valid_0's multi_logloss: 0.395897
[48]	valid_0's multi_logloss: 0.394357
[49]	valid_0's multi_logloss: 0.393688
[50]	valid_0's multi_logloss: 0.392251
[51]	valid_0's multi_logloss: 0.390817
Did not meet early stopping. Best iteration is:
[51]	valid_0's multi_logloss: 0.390817
[1]	valid_0's multi_logloss: 0.704851
Training until validation scores don't improve for 3 rounds.
[2]	valid_0's multi_logloss: 0.672928
[3]	valid_0's multi_logloss: 0.6446
[4]	valid_0's multi_logloss: 0.62051
[5]	valid_0's multi_logloss: 0.599766
[6]	valid_0's multi_logloss: 0.585128
[7]	valid_0's multi_logloss: 0.570439
[8]	valid_0's multi_logloss: 0.557769
[9]	valid_0's multi_logloss: 0.544028
[10]	valid_0's multi_logloss: 0.531767
[11]	valid_0's multi_logloss: 0.521529
[12]	valid_0's multi_logloss: 0.511881
[13]	valid_0's mul

In [39]:
mejores_hp.best_params_

{'colsample_bylevel': 0.6499999999999999,
 'colsample_bytree': 0.7499999999999999,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.22,
 'max_depth': 5,
 'min_child_weight': 5,
 'n_estimators': 91,
 'scale_pos_weight': 4,
 'subsample': 0.5}

In [40]:
predicciones_mejores_hp_train = mejores_hp.predict_proba(X_test)

In [41]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_mejores_hp_train)

0.4077233684542726

In [42]:
predicciones_mejores_hp = mejores_hp.predict_proba(test_set_features[['Account_Owner',
                            'Opportunity_Owner',
                            'Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']])

Nos quedamos unicamente con las probabilidades de exito


In [43]:
predicciones_mejores_hp_exito = []
for prob in predicciones_mejores_hp:
  predicciones_mejores_hp_exito.append(prob[1])

In [44]:
csv_kaggle_mejores_hp = pd.DataFrame()
csv_kaggle_mejores_hp['Opportunity_ID'] = test_set_features["Opportunity_ID"]
csv_kaggle_mejores_hp['target'] = predicciones_mejores_hp_exito

In [46]:
from google.colab import files
csv = csv_kaggle_mejores_hp.to_csv('predicciones_lgb_random_search_unico_reg_por_op_11.csv', index=False)
files.download('predicciones_lgb_random_search_unico_reg_por_op_11.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Resultados 

Algoritmo: LightGBM

Features analizados:

+ Total_Taxable_Amount_USD
+ Products_Per_Op
+ TRF_Per_Op
+ Account_Owner
+ Bureaucratic_Code

Hiperparámetros:

Metodo: Random Search

Resultado kaggle = 0.53377

Resultados 2 

Algoritmo: LightGBM

Features analizados:

+ Account_Owner 
+ Total_Taxable_Amount_USD 
+ Bureaucratic_Code 
+ Products_Per_Op 
+ TRF_Per_Op 
+ Daily_Dif_Account_Created_Opportunity_Created_Date 
+ Daily_Dif_Delivery_Start_Delivery_End_Date 
+ Opp_Owner_Did_Last_Modifiy

Hiperparámetros:

Metodo: Random Search

Resultado kaggle = 0.51056

Resultados 3

Algoritmo: LightGBM

Features analizados:

+ Account_Owner 
+ Total_Taxable_Amount_USD 
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op 
+ TRF_Per_Op 
+ Daily_Dif_Account_Created_Opportunity_Created_Date 
+ Daily_Dif_Delivery_Start_Delivery_End_Date 
+ Opp_Owner_Did_Last_Modifiy
+ Pricing, Delivery_Terms_Approved
+ Bureaucratic_Code_0_Approval

Hiperparámetros:

'colsample_bylevel': 0.7500000000000001,
 'colsample_bytree': 0.7500000000000001,
 'gamma': 0.1,
 'learning_rate': 0.30000000000000004,
 'max_depth': 4,
 'min_child_weight': 4,
 'n_estimators': 76,
 'scale_pos_weight': 3,
 'subsample': 0.7

Metodo: Random Search

Resultado kaggle = 0.49556

Resultados 4

Algoritmo: LightGBM

Features analizados:

+ Account_Owner 
+ Total_Taxable_Amount_USD 
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op 
+ TRF_Per_Op 
+ Daily_Dif_Account_Created_Opportunity_Created_Date 
+ Daily_Dif_Delivery_Start_Delivery_End_Date 
+ Opp_Owner_Did_Last_Modifiy 
+ Bureaucratic_Code_0_Approval 
+ Pricing, Delivery_Terms_Approved 
+ Op_Has_A_Valid_Quote_Expiry_Date 
+ Last_Modified_date>Planned_Delivery_End_Date

Hiperparámetros:

Metodo: Random Search

{'colsample_bylevel': 0.8000000000000002,
 'colsample_bytree': 0.9000000000000002,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.4,
 'max_depth': 4,
 'min_child_weight': 3,
 'n_estimators': 91,
 'scale_pos_weight': 5,
 'subsample': 0.6}

Resultado kaggle = 0.50172

Resultados 5

Algoritmo: LightGBM

Features analizados:

Account_Owner
Total_Taxable_Amount_USD
Bureaucratic_Code_0_Approved
Products_Per_Op
TRF_Per_Op
Daily_Dif_Account_Created_Opportunity_Created_Date
Daily_Dif_Delivery_Start_Delivery_End_Date
Opp_Owner_Did_Last_Modifiy
Bureaucratic_Code_0_Approval
Pricing, Delivery_Terms_Approved
Op_Has_A_Valid_Quote_Expiry_Date
Last_Modified_date>Planned_Delivery_End_Date
Hiperparámetros:

Metodo: Random Search

{'colsample_bylevel': 0.7500000000000001,
 'colsample_bytree': 0.7500000000000001,
 'gamma': 0.14999999999999997,
 'learning_rate': 0.30000000000000004,
 'max_depth': 4,
 'min_child_weight': 4,
 'n_estimators': 76,
 'scale_pos_weight': 3,
 'subsample': 0.76}

Resultado kaggle = 0.49506

Resultados 6

Algoritmo: LightGBM

Features analizados:

Account_Owner
Total_Taxable_Amount_USD
Bureaucratic_Code_0_Approved
Products_Per_Op
TRF_Per_Op
Daily_Dif_Account_Created_Opportunity_Created_Date
Daily_Dif_Delivery_Start_Delivery_End_Date
Opp_Owner_Did_Last_Modifiy
Bureaucratic_Code_0_Approval
Pricing, Delivery_Terms_Approved
Op_Has_A_Valid_Quote_Expiry_Date
Last_Modified_date>Planned_Delivery_End_Date
Hiperparámetros:

Metodo: Random Search

Obs: utilizamos 10000 iteraciones

{'colsample_bylevel': 0.35,
 'colsample_bytree': 0.7,
 'gamma': 0.0,
 'learning_rate': 0.26,
 'max_depth': 4,
 'min_child_weight': 6,
 'n_estimators': 111,
 'scale_pos_weight': 2,
 'subsample': 0.7}

Resultado kaggle = 0.48806

Agregando columnas obtenidas con Feature Engineering, los resultados mejoraron.